---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [2]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [3]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [4]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    df = pd.read_table('university_towns.txt',sep='\n',header=None)

    #build dataframe 

    st = pd.DataFrame(columns = ['State','RegionName'])
    for index, row in df.iterrows():
            if df[0].str.endswith('[edit]')[index]:
                State = df.iloc[index].str.split('[')[0][0]
            else:
                st.loc[index] = [State,df.iloc[index].str.split('(')[0][0]]
                

    st['RegionName'] = st['RegionName'].str.strip()

    return st

get_list_of_university_towns()

,State,RegionName
1,Alabama,Auburn
2,Alabama,Florence
3,Alabama,Jacksonville
4,Alabama,Livingston
5,Alabama,Montevallo
...,...,...
561,Wisconsin,River Falls
562,Wisconsin,Stevens Point
563,Wisconsin,Waukesha
564,Wisconsin,Whitewater


In [5]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    
    pd.options.display.max_rows = 10
    
    
    #read in GDP
    gdp = pd.read_excel('gdplev.xls',usecols="E:G", skiprows=7,names=('Quarter','GDP','GDP Chained')) 
    gdp = gdp.iloc[-66:,]
    #gdp['Quarter'] = pd.to_datetime(gdp['Quarter']) 
    
    #calculate change in Chained GDP 
    gdp['Delta'] = gdp['GDP Chained'].diff()

    #Calculate begin & end quarters of recessions
    begin = (gdp['Delta']<0) & (gdp['Delta'].shift(-1)<0)  # potential start
    end   = (gdp['Delta']>0) & (gdp['Delta'].shift(-1)>0)  # potential end

    gdp['Recession' ] = np.nan
    gdp.loc[begin, 'Recession'] = -1
    gdp.loc[end, 'Recession'] =  1
    
    #fill in recession or recovery values when status doesn't change
    gdp['Recession'] = gdp.Recession.ffill()                                                                          
    gdp['ChangeOver'] = np.nan
    gdp.loc[ (gdp.Recession==-1) & (gdp.Recession.shift()== 1), 'ChangeOver'] = -1  # begin
    gdp.loc[ (gdp.Recession== 1) & (gdp.Recession.shift()==-1), 'ChangeOver'] =  1  # end

    gdp.set_index(['Quarter'])
    
    return gdp[gdp.Recession==-1].min()['Quarter']#gdp[gdp.ChangeOver.notnull()].loc[gdp['Recession']==-1,'Quarter'].iloc[0]

get_recession_start()

'2008q3'

In [6]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    pd.options.display.width = 0
    #read in GDP
    gdp = pd.read_excel('gdplev.xls',usecols="E:G", skiprows=7,names=('Quarter','GDP','GDP Chained')) 
    gdp = gdp.iloc[-66:,]
    #gdp['Quarter'] = pd.to_datetime(gdp['Quarter']) 
    #calculate change in Chained GDP 
    gdp['Delta'] = gdp['GDP Chained'].diff()

    #Calculate begin & end quarters of recessions
    begin = (gdp['Delta']<0) & (gdp['Delta'].shift(-1)<0)  # potential start
    end   = (gdp['Delta'].shift()>0) & (gdp['Delta'].shift(-1)>0)  # potential end

    gdp['Recession' ] = np.nan
    gdp.loc[begin, 'Recession'] = -1
    gdp.loc[end, 'Recession'] =  1
    
    #fill in recession or recovery values when status doesn't change
    gdp['Recession'] = gdp.Recession.ffill()                             
                                                                              
    gdp['ChangeOver'] = np.nan
    gdp.loc[ (gdp.Recession==-1) & (gdp.Recession.shift()== 1), 'ChangeOver'] = -1  # begin
    gdp.loc[ (gdp.Recession== 1) & (gdp.Recession.shift()==-1), 'ChangeOver'] =  1  # end
    

    return gdp.loc[(gdp['Recession']==1) & (gdp['ChangeOver']==1)].iloc[0,0]
                                                                       
    

get_recession_end()

'2009q4'

In [7]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3''' 
    #read in GDP
    gdp = pd.read_excel('gdplev.xls',usecols="E:G", skiprows=7,names=('Quarter','GDP','GDP Chained')) 
    gdp = gdp.iloc[-66:,]
    
    #calculate change in Chained GDP 
    gdp['Delta'] = gdp['GDP Chained'].diff()

    #Calculate begin & end quarters of recessions
    begin = (gdp['Delta']<0) & (gdp['Delta'].shift(-1)<0)
    end   = (gdp['Delta']>0) & (gdp['Delta'].shift(-1)>0)

    gdp['Recession' ] = np.nan
    gdp.loc[begin, 'Recession'] = -1
    gdp.loc[end, 'Recession'] =  1
    
    #fill in recession or recovery values when status doesn't change
    gdp['Recession'] = gdp.Recession.ffill()                             
                                                                              
    gdp['ChangeOver'] = np.nan
    gdp.loc[(gdp.Recession==-1) & (gdp.Recession.shift()== 1), 'ChangeOver'] = -1  # begin
    gdp.loc[(gdp.Recession== 1) & (gdp.Recession.shift()==-1), 'ChangeOver'] =  1  # end
    
    #gdp['Quarter'] = pd.to_datetime(gdp['Quarter']) 
    mask = (gdp['Quarter'] >= get_recession_start()) & (gdp['Quarter'] <= get_recession_end())

    
    #gdp_recession = gdp[gdp.Quarter>get_recession_start() & gdp.Quarter < get_recession_end()]
    return gdp['Quarter'][gdp.loc[mask]['GDP Chained'].idxmin()]#gdp_recession #gdp['Quarter'][gdp['GDP Chained'].idxmin()]

get_recession_bottom()


'2009q2'

In [8]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    pd.options.display.max_rows = 1000
    hp = pd.read_csv('City_Zhvi_AllHomes.csv')
    hp['State'] = hp['State'].map(states)
    hp=hp.iloc[:,np.r_[[1,2],51:251]]
    hp.set_index(['State','RegionName'],inplace=True)
    hp.sort_index(inplace=True)
    hp=hp.groupby(pd.PeriodIndex(hp.columns, freq='Q'), axis=1).mean()
        

    #df=df.assign(df.iloc[:,4:].groupby(pd.PeriodIndex(df.iloc[:,4:].columns, freq='Q'), axis=1).mean())
    #st = df.iloc[:,[1,2]]
    #house = pd.concat([st,quarters])
    #pd.MultiIndex.from_frame(house.iloc[:,[0,1]]
    return hp #.iloc[:,4:].groupby(pd.PeriodIndex(df.iloc[:,4:].columns, freq='Q'), axis=1).mean()

convert_housing_data_to_quarters()

2000Q1         2000Q2         2000Q3  \
State   RegionName                                                 
Alabama Adamsville    69033.333333   69166.666667   69800.000000   
        Alabaster    122133.333333  123066.666667  123166.666667   
        Albertville   73966.666667   72600.000000   72833.333333   
        Arab          83766.666667   81566.666667   81333.333333   
        Ardmore                NaN            NaN            NaN   
...                            ...            ...            ...   
Wyoming Burns        101533.333333  104566.666667  108366.666667   
        Casper        89233.333333   89600.000000   89733.333333   
        Cheyenne     116866.666667  120033.333333  121533.333333   
        Evansville   128033.333333  128766.666667  130833.333333   
        Pine Bluffs   93733.333333   95066.666667   94633.333333   

                            2000Q4         2001Q1         2001Q2  \
State   RegionName                                                 
Alabama Adamsville    71966.666667   73466.666667   74000.000000   
        Alabaster    123700.000000  123233.333333  125133.333333   
        Albertville   74200.000000   75900.000000   76000.000000   
        Arab          82966.666667   84200.000000   84533.333333   
        Ardmore                NaN            NaN            NaN   
...                            ...            ...            ...   
Wyoming Burns        113000.000000  115833.333333  117200.000000   
        Casper        93166.666667   95500.000000   97633.333333   
        Cheyenne     123633.333333  125533.333333  126300.000000   
        Evansville   132066.666667  130566.666667  131433.333333   
        Pine Bluffs   98066.666667  103233.333333  104600.000000   

                            2001Q3         2001Q4         2002Q1  \
State   RegionName                                                 
Alabama Adamsville    73333.333333   73100.000000   73333.333333   
        Alabaster    127766.666667  127200.000000  127300.000000   
        Albertville   72066.666667   73566.666667   76533.333333   
        Arab          81666.666667   83900.000000   87266.666667   
        Ardmore                NaN            NaN            NaN   
...                            ...            ...            ...   
Wyoming Burns        117800.000000  117633.333333  117333.333333   
        Casper        99433.333333  100633.333333  101733.333333   
        Cheyenne     126466.666667  128133.333333  128466.666667   
        Evansville   132400.000000  133466.666667  133300.000000   
        Pine Bluffs  106500.000000  104066.666667  102233.333333   

                            2002Q2  ...         2014Q2         2014Q3  \
State   RegionName                  ...                                 
Alabama Adamsville    73133.333333  ...   77066.666667   75966.666667   
        Alabaster    128000.000000  ...  147133.333333  147633.333333   
        Albertville   76366.666667  ...   84033.333333   84766.666667   
        Arab          87700.000000  ...  113366.666667  111700.000000   
        Ardmore                NaN  ...  140533.333333  139566.666667   
...                            ...  ...            ...            ...   
Wyoming Burns        117233.333333  ...  168866.666667  161933.333333   
        Casper       101533.333333  ...  175766.666667  177300.000000   
        Cheyenne     129633.333333  ...  177466.666667  176733.333333   
        Evansville   131066.666667  ...  296733.333333  305666.666667   
        Pine Bluffs  103566.666667  ...  148666.666667  154366.666667   

                            2014Q4         2015Q1         2015Q2  \
State   RegionName                                                 
Alabama Adamsville    71900.000000   71666.666667   73033.333333   
        Alabaster    148700.000000  148900.000000  149566.666667   
        Albertville   86800.000000   88466.666667   89500.000000   
        Arab         111600.000000  110166.666667  109433.333333   
        Ardmore

In [12]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    from scipy import stats
    from pandas.tseries.offsets import DateOffset
    
    hp=convert_housing_data_to_quarters()
    rb=get_recession_bottom()
    re=get_recession_end()
    rs=get_recession_start()
    uni = get_list_of_university_towns()
    
    #Process Uni to Join
    uni['Indicator'] = 'university town'
    uni.set_index(['State','RegionName'],inplace=True)
    uni.sort_index(inplace=True)
    
    #build a datagframe by town, with a binary field for university town
    #

    df = pd.DataFrame(index=hp.index,columns = ['HVI Start', 'HVI Bottom', 'HVI Change'])
    df['HVI Start'] = hp['2008Q2']
    df['HVI Bottom'] = hp['2009Q2']
    df['HVI Change'] = df['HVI Start'].div(df['HVI Bottom'])
    
    
    df.sort_index(inplace=True)
    uni.sort_index(inplace=True)
    
    #Join College with Household 
    df = pd.merge(df,uni, left_on=['State', 'RegionName'],right_on=['State', 'RegionName'],how='left')

    df['Indicator']=df['Indicator'].fillna('non-university town')

    #run t test
  
    nonU = df[df['Indicator']=='non-university town']['HVI Change']
    U = df[df['Indicator']=='university town']['HVI Change']
    different = 'False'
    better = 'non-university town'    
    p = stats.ttest_ind(nonU,U,nan_policy='omit')[1]     
    
    if p <= 0.05:
        different = bool('True')
                                                    
    if U.mean()<nonU.mean():
        better = 'university town'
                                                      
                                                    
    #answer=tuple(['True',[stats.ttest_ind(U, nonU,equal_var=False)[1],nan_policy='omit'],'university town')
    
    #tuple(['True',[stats.ttest_ind(nonU,U,nan_policy='omit')[1]],'university town'])
    
    return tuple([different,p,better])

run_ttest()

(True, 0.002724063704753125, 'university town')

HVI Start     HVI Bottom    HVI Change  \
State   RegionName                                                
Alabama Adamsville    86066.666667   85766.666667   -300.000000   
        Alabaster    156933.333333  149666.666667  -7266.666667   
        Albertville   87933.333333   84333.333333  -3600.000000   
        Arab         111033.333333  115366.666667   4333.333333   
        Ardmore      149866.666667  143900.000000  -5966.666667   
...                            ...            ...           ...   
Wyoming Burns        169466.666667  209366.666667  39900.000000   
        Casper       198033.333333  196533.333333  -1500.000000   
        Cheyenne     182833.333333  193533.333333  10700.000000   
        Evansville   259400.000000  279266.666667  19866.666667   
        Pine Bluffs  156733.333333  186433.333333  29700.000000   

                               Indicator  
State   RegionName                        
Alabama Adamsville   non-university town  
        Alabaster    non-university town  
        Albertville  non-university town  
        Arab         non-university town  
        Ardmore      non-university town  
...                                  ...  
Wyoming Burns        non-university town  
        Casper       non-university town  
        Cheyenne     non-university town  
        Evansville   non-university town  
        Pine Bluffs  non-university town  

[10730 rows x 4 columns]

In [279]:
get_list_of_university_towns().groupby('State').size()

State
Alabama            8
Alaska             1
Arizona            3
Arkansas           8
California        25
Colorado           9
Connecticut        7
Delaware           2
Florida           12
Georgia           16
Hawaii             1
Idaho              3
Illinois          10
Indiana           13
Iowa              13
Kansas             6
Kentucky          11
Louisiana          8
Maine              9
Maryland          10
Massachusetts     19
Michigan          20
Minnesota         14
Mississippi        5
Missouri           9
Montana            3
Nebraska           7
Nevada             2
New Hampshire      7
New Jersey        10
New Mexico         5
New York          31
North Carolina    18
North Dakota       2
Ohio              22
Oklahoma          11
Oregon            10
Pennsylvania      43
Rhode Island       2
South Carolina    11
South Dakota       4
Tennessee         13
Texas             22
Utah               6
Vermont            6
Virginia          12
Washington         5
West Vi